In [79]:
import os
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import cohen_kappa_score, accuracy_score, mean_absolute_error, f1_score
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
from tqdm import tqdm
import xgboost as xgb
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import gc
from datetime import datetime

warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [80]:
seed = 2020

In [81]:
df_feature = pd.read_csv('processed_data.csv')

In [82]:
from sklearn.preprocessing import LabelEncoder
for f in tqdm(df_feature.select_dtypes('object')):
    lbl = LabelEncoder()
    df_feature[f] = lbl.fit_transform(df_feature[f].astype(str))

  0%|          | 0/4424 [00:00<?, ?it/s]


In [83]:
df_test = df_feature[df_feature['label']==1].drop(columns=['label']).copy()
df_acc = df_feature[df_feature['label']==2].drop(columns=['label']).copy()
df_train = df_feature.copy()

In [84]:
df_test.head()

,date,Latitude,Longitude,notes_polarity,notes_subjectivity,comments_polarity,comments_subjectivity,best_season
308,198,47.487496,-118.958142,0.0,0.0,-0.5,0.5,0
309,135,47.574972,-117.678159,0.0,0.0,0.0,0.5,0
310,161,48.016374,-122.053332,0.0,0.0,0.0,0.0,0
311,161,46.172385,-122.914926,0.0,0.0,0.0,0.0,0
312,187,46.940812,-119.963979,0.0,0.0,0.0,0.0,0


In [85]:
df_train.head()

,date,label,Latitude,Longitude,notes_polarity,notes_subjectivity,comments_polarity,comments_subjectivity,best_season
0,193,2,47.450510,-120.359030,0.0,0.0,0.0,0.0,0
1,220,2,46.210088,-119.759287,0.0,0.0,0.2,0.2,1
2,235,2,47.629840,-120.689450,0.0,0.0,0.2,0.2,1
3,210,2,47.445464,-120.267607,0.0,0.0,0.2,0.2,0
4,197,2,47.143276,-119.852737,0.0,0.0,0.0,0.0,0


In [86]:
df_acc.head()

,date,Latitude,Longitude,notes_polarity,notes_subjectivity,comments_polarity,comments_subjectivity,best_season
0,193,47.450510,-120.359030,0.0,0.0,0.0,0.0,0
1,220,46.210088,-119.759287,0.0,0.0,0.2,0.2,1
2,235,47.629840,-120.689450,0.0,0.0,0.2,0.2,1
3,210,47.445464,-120.267607,0.0,0.0,0.2,0.2,0
4,197,47.143276,-119.852737,0.0,0.0,0.0,0.0,0


In [87]:
ycol = 'label'
feature_names = list(
    filter(lambda x: x not in [ycol], df_train.columns))

# model = xgb.XGBClassifier(min_child_weight=6,max_depth=15,
#                                 objective='multi:softmax',num_class=5)
model = xgb.XGBRegressor(num_leaves=64,
                         max_depth=8,
                         learning_rate=0.08,
                         n_estimators=10000000,
                         subsample=0.75,
                         feature_fraction=0.75,
                         reg_alpha=0.7,
                         reg_lambda=1.2,
                         random_state=seed,
                         metric=None,
                         tree_method='hist'
                         )
oof = []
prediction = df_test.copy()
prediction['label'] = 0
predictionacc = df_acc.copy()
predictionacc['label'] = 0
df_importance_list = []

kfold = KFold(n_splits=5, shuffle=True, random_state=seed)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(df_train[feature_names])):
    X_train = df_train.iloc[trn_idx][feature_names]
    Y_train = df_train.iloc[trn_idx][ycol]

    X_val = df_train.iloc[val_idx][feature_names]
    Y_val = df_train.iloc[val_idx][ycol]

    print('\nFold_{} Training ================================\n'.format(fold_id+1))

    # xgb_model = model.fit(X_train,
    #                       Y_train,
    #                       eval_set=[(X_train, Y_train), (X_val, Y_val)],
    #                       verbose=1000,
    #                       eval_metric="merror",
    #                       early_stopping_rounds=500)
    xgb_model = model.fit(X_train,
                          Y_train,
                          eval_set=[(X_train, Y_train), (X_val, Y_val)],
                          verbose=1000,
                          eval_metric='mae',
                          early_stopping_rounds=500)

    pred_val = xgb_model.predict(
        X_val)
    df_oof = df_train.iloc[val_idx].copy()
    df_oof['label'] = pred_val
    oof.append(df_oof)

    pred_test = xgb_model.predict(
        df_test[feature_names])
    prediction['label'] += pred_test / 5

    pred_acc = xgb_model.predict(
        df_acc[feature_names])
    predictionacc['label'] += pred_acc / 5

    df_importance = pd.DataFrame({
        'column': feature_names,
        'importance': xgb_model.feature_importances_,
    })
    df_importance_list.append(df_importance)

    del xgb_model, pred_val, pred_test, pred_acc, X_train, Y_train, X_val, Y_val
    gc.collect()



Fold_1 Training ================================

[20:58:47] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1643226991592/work/src/learner.cc:576: 
Parameters: { "feature_fraction", "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-mae:0.90752	validation_1-mae:0.90126
[573]	validation_0-mae:0.04038	validation_1-mae:0.20201

Fold_2 Training ================================

[20:58:57] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1643226991592/work/src/learner.cc:576: 
Parameters: { "feature_fraction", "num_leaves" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter act

In [88]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby(['column'])['importance'].agg(
    'mean').sort_values(ascending=False).reset_index()
df_importance

,column,importance
0,comments_subjectivity,0.691406
1,date,0.063807
2,best_season,0.060603
3,Latitude,0.043966
4,Longitude,0.037702
5,notes_subjectivity,0.035330
6,notes_polarity,0.033661
7,comments_polarity,0.033525


In [89]:
df_oof = pd.concat(oof)
df_oof[ycol] = np.expm1(df_oof[ycol])
df_oof['label'] = np.expm1(df_oof['label'])
# df_oof.to_csv('xgb_oof.csv'.format(mae), index=False, encoding='utf-8')

In [90]:
prediction['label'] = np.expm1(prediction['label'])
sub = prediction.copy(deep=True)
sub.to_csv('res.csv', index=False, encoding='utf-8')

In [91]:
predictionacc['label'] = np.expm1(predictionacc['label'])
sub = predictionacc.copy(deep=True)
sub.to_csv('acc.csv', index=False, encoding='utf-8')

In [92]:
sub.head()

,date,Latitude,Longitude,notes_polarity,notes_subjectivity,comments_polarity,comments_subjectivity,best_season,label
0,193,47.450510,-120.359030,0.0,0.0,0.0,0.0,0,4.772114
1,220,46.210088,-119.759287,0.0,0.0,0.2,0.2,1,6.395565
2,235,47.629840,-120.689450,0.0,0.0,0.2,0.2,1,6.354036
3,210,47.445464,-120.267607,0.0,0.0,0.2,0.2,0,6.293356
4,197,47.143276,-119.852737,0.0,0.0,0.0,0.0,0,5.067774
